# Vulnerability Score Analysis

This notebook processes census sections (seccions censals) from Barcelona and assigns weather stations to each section based on proximity.

In [1]:
import geopandas as gpd
import pandas as pd


## 1. Load Census Sections Data

Load the Barcelona census sections from CSV. The file contains polygon geometries in WKT format (WGS84 coordinate system).


In [2]:
# Load CSV with WKT geometry
df = gpd.read_file("data/BarcelonaCiutat_SeccionsCensals.csv", GEOM_POSSIBLE_NAMES="geometria_wgs84", KEEP_GEOM_COLUMNS="NO")

# Create GeoDataFrame with geometry
gdf = gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")

# Calculate centroids (for distance calculations later)
# Note: For accurate centroid calculations, we'll reproject to UTM in a later step
gdf["centroid"] = gdf.geometry.centroid

# Extract latitude and longitude from centroids
gdf["centroid_lat"] = gdf["centroid"].y
gdf["centroid_lon"] = gdf["centroid"].x

gdf.head()

/var/folders/gy/5nns71_n7nx_v_2mkns5_k340000gn/T/ipykernel_34568/2562052214.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf["centroid"] = gdf.geometry.centroid


,codi_districte,nom_districte,codi_barri,nom_barri,codi_aeb,codi_seccio_censal,geometria_etrs89,geometry,centroid,centroid_lat,centroid_lon
0,01,Ciutat Vella,01,el Raval,001,001,"POLYGON ((431076.9025 4581077.3095, 431058.164...","POLYGON ((2.17575 41.37827, 2.17552 41.37865, ...",POINT (2.17722 41.37432),41.374324,2.177219
1,01,Ciutat Vella,01,el Raval,001,002,"POLYGON ((431023.5455 4581164.3265, 430990.550...","POLYGON ((2.1751 41.37905, 2.1747 41.37951, 2....",POINT (2.17391 41.37793),41.377927,2.173914
2,01,Ciutat Vella,01,el Raval,002,003,"POLYGON ((430778.3455 4580930.5395, 430766.851...","POLYGON ((2.1722 41.37692, 2.17206 41.37696, 2...",POINT (2.17199 41.37576),41.375757,2.171985
3,01,Ciutat Vella,01,el Raval,002,004,"POLYGON ((430564.2645 4581104.2995, 430496.863...","POLYGON ((2.16962 41.37847, 2.16882 41.37784, ...",POINT (2.16924 41.37642),41.376416,2.169238
4,01,Ciutat Vella,01,el Raval,003,005,"POLYGON ((430905.0315 4581350.0725, 430874.963...","POLYGON ((2.17366 41.38071, 2.1733 41.38113, 2...",POINT (2.17277 41.37884),41.378836,2.172773


## 2. Create SECCIO_CENSAL Identifier

Create a unique identifier for each census section by concatenating:
- Prefix: `080193` (Barcelona municipality code)
- District code: `codi_districte`
- Section code: `codi_seccio_censal`

This creates a standardized identifier format: `080193 + codi_districte + codi_seccio_censal`


In [3]:
# Create SECCIO_CENSAL field: prefix 080193 + codi_districte + codi_seccio_censal
gdf["SECCIO_CENSAL"] = "080193" + gdf["codi_districte"].astype(str) + gdf["codi_seccio_censal"].astype(str)

# Verify the identifier creation
gdf[["codi_districte", "codi_seccio_censal", "SECCIO_CENSAL"]].head()


,codi_districte,codi_seccio_censal,SECCIO_CENSAL
0,01,001,08019301001
1,01,002,08019301002
2,01,003,08019301003
3,01,004,08019301004
4,01,005,08019301005


In [4]:
# Verify uniqueness of SECCIO_CENSAL
num_unique = gdf["SECCIO_CENSAL"].nunique()
print(f"Number of unique values in SECCIO_CENSAL: {num_unique}")
print(f"Total number of rows: {len(gdf)}")


Number of unique values in SECCIO_CENSAL: 1068
Total number of rows: 1068


**Note:** This corresponds to the number of census sections in Barcelona (1068), confirming that our identifier field is correct and unique for each section.

## 3. Load Weather Stations

Load the weather stations for which we have complete data for 2023 and 2024. 

**Note:** We exclude the Zoo weather station (X2) for our simplified version of the map, since it was dismantled in October 2024.

The three stations used are:
- **D5**: Located at coordinates (2.12379, 41.41864)
- **X4**: Located at coordinates (2.16775, 41.38390)
- **X8**: Located at coordinates (2.10540, 41.37919)


In [5]:
# Create GeoDataFrame with weather station locations
stations = gpd.GeoDataFrame(
    {
        "name": ["D5", "X4", "X8"],
        "lat": [41.41864, 41.38390, 41.37919],
        "lon": [2.12379, 2.16775, 2.10540],
    },
    geometry=gpd.points_from_xy(
        [2.12379, 2.16775, 2.10540],  # longitude (x)
        [41.41864, 41.38390, 41.37919],  # latitude (y)
    ),
    crs="EPSG:4326"  # WGS84 coordinate system
)

stations

,name,lat,lon,geometry
0,D5,41.41864,2.12379,POINT (2.12379 41.41864)
1,X4,41.38390,2.16775,POINT (2.16775 41.3839)
2,X8,41.37919,2.10540,POINT (2.1054 41.37919)


## 4. Assign Weather Stations to Census Sections

For each census section, we find the nearest weather station using spatial joins. This process involves:

1. **Reproject to UTM (EPSG:25831)**: Convert both datasets to a projected coordinate system (UTM Zone 31N) for accurate distance calculations in meters
2. **Calculate centroids**: Compute the centroid of each census section polygon in the projected CRS
3. **Find nearest station**: Use `sjoin_nearest` to find the closest weather station to each centroid
4. **Merge results**: Add the assigned station name and distance to the original GeoDataFrame


In [6]:
# Step 1: Reproject to UTM for accurate distance calculations
gdf_utm = gdf.to_crs(epsg=25831)  # UTM Zone 31N (Spain)
stations_utm = stations.to_crs(epsg=25831)

# Step 2: Calculate centroids in projected CRS (more accurate than WGS84)
gdf_utm["centroid"] = gdf_utm.geometry.centroid
centroids = gdf_utm.set_geometry("centroid")

# Step 3: Find nearest weather station to each census section centroid
nearest = gpd.sjoin_nearest(
    centroids,
    stations_utm[["name", "geometry"]],
    how="left",
    distance_col="dist_m"  # Distance in meters
)

# Step 4: Merge results back to original GeoDataFrame
# Merge on index since sjoin_nearest preserves the index from centroids
gdf = gdf.merge(
    nearest[["name", "dist_m"]],
    left_index=True,
    right_index=True,
    how="left"
)
gdf = gdf.rename(columns={"name": "WEATHER_STATION"})

# Display results
print(f"Number of census sections assigned to stations: {gdf['WEATHER_STATION'].notna().sum()}")
print(f"\nDistribution of stations:")
print(gdf["WEATHER_STATION"].value_counts())
print(f"\nDistance statistics (meters):")
print(gdf["dist_m"].describe())

Number of census sections assigned to stations: 1068

Distribution of stations:
WEATHER_STATION
X4    625
D5    338
X8    105
Name: count, dtype: int64

Distance statistics (meters):
count    1068.000000
mean     2988.696968
std      1492.095398
min       121.156567
25%      1914.985512
50%      2611.524009
75%      4095.279686
max      7152.228289
Name: dist_m, dtype: float64


## 5. Preview Results

Preview the final GeoDataFrame with assigned weather stations:


In [7]:
# Preview the final GeoDataFrame
gdf[["SECCIO_CENSAL", "nom_districte", "nom_barri", "WEATHER_STATION", "dist_m"]].head(10)

,SECCIO_CENSAL,nom_districte,nom_barri,WEATHER_STATION,dist_m
0,08019301001,Ciutat Vella,el Raval,X4,1325.662820
1,08019301002,Ciutat Vella,el Raval,X4,839.842507
2,08019301003,Ciutat Vella,el Raval,X4,970.927625
3,08019301004,Ciutat Vella,el Raval,X4,840.155800
4,08019301005,Ciutat Vella,el Raval,X4,701.768850
5,08019301006,Ciutat Vella,el Raval,X4,544.606912
6,08019301007,Ciutat Vella,el Raval,X4,560.631260
7,08019301008,Ciutat Vella,el Raval,X4,675.929543
8,08019301009,Ciutat Vella,el Raval,X4,726.465578
9,08019301010,Ciutat Vella,el Raval,X4,563.659021


## 6. Merge Daily Weather Data with Census Sections

Load the cleaned weather data and merge it with census sections. The weather data is in long format (one row per station-date-variable), so we need to pivot it to wide format (one row per station-date with columns for each variable).

### Date Range Filtering

For simplicity, we filter the weather data to the period from **2023-01-04 to 2024-12-31**. This represents the temporal overlap between all our data sources:
- **Weather data**: Available from 2021-01-01 to 2025-11-16
- **Consumption data**: Available from 2023-01-04 onwards
- **Leak incidents**: Available from 2023 onwards

By focusing on this common period, we ensure all data sources are available for analysis while maintaining a substantial time range for our vulnerability score calculations.


In [14]:
# Load weather data and filter to only the stations we use (D5, X4, X8)
weather = pd.read_parquet("clean/weather_clean.parquet")

# Filter to only the stations assigned to census sections
stations_to_keep = ['D5', 'X4', 'X8']
weather = weather[weather['CODI_ESTACIO'].isin(stations_to_keep)].copy()

# Drop NOM_ESTACIO since we already have CODI_ESTACIO
if 'NOM_ESTACIO' in weather.columns:
    weather = weather.drop(columns=['NOM_ESTACIO'])

# Filter to date range: 2023-01-04 to 2024-12-31 (temporal overlap with consumption and leaks)
date_start = pd.to_datetime('2023-01-04')
date_end = pd.to_datetime('2024-12-31')
weather = weather[
    (pd.to_datetime(weather['DATA_LECTURA']) >= date_start) & 
    (pd.to_datetime(weather['DATA_LECTURA']) <= date_end)
].copy()

print(f"Weather data shape (after filtering): {weather.shape}")
print(f"\nStations in weather data: {sorted(weather['CODI_ESTACIO'].unique())}")
print(f"\nDate range: {weather['DATA_LECTURA'].min()} to {weather['DATA_LECTURA'].max()}")
print(f"\nNumber of variables: {weather['NOM_VARIABLE'].nunique()}")
print(f"\nSample of weather data:")
weather.head()


Weather data shape (after filtering): (48021, 10)

Stations in weather data: ['D5', 'X4', 'X8']

Date range: 2023-01-04 00:00:00 to 2024-12-31 00:00:00

Number of variables: 22

Sample of weather data:


,ID,CODI_ESTACIO,DATA_LECTURA,CODI_VARIABLE,NOM_VARIABLE,VALOR,UNITAT,HORA _TU,Estat,VALOR_NUM
54232,D51000012304,D5,2023-01-04,1.000,Temperatura mitjana diària,"11,7",°C,NA,Representatiu,11.7
54233,D51001012304,D5,2023-01-04,1.001,Temperatura màxima diària + hora,"17,7",°C,14:14:00,Representatiu,17.7
54234,D51002012304,D5,2023-01-04,1.002,Temperatura mínima diària + hora,"9,1",°C,06:06:00,Representatiu,9.1
54235,D51003012304,D5,2023-01-04,1.003,Temperatura mitjana diària clàssica,"13,4",°C,NA,Representatiu,13.4
54236,D51004012304,D5,2023-01-04,1.004,Amplitud tèrmica diària,"8,6",°C,NA,Representatiu,8.6


In [15]:
# Pivot weather data from long to wide format
# Each row will be a unique combination of station (CODI_ESTACIO) and date (DATA_LECTURA)
# Each variable (NOM_VARIABLE) becomes a column with its VALOR_NUM value
# Note: No duplicate measurements - each station-date-variable has only one value

weather_daily = weather.pivot_table(
    index=['CODI_ESTACIO', 'DATA_LECTURA'],
    columns='NOM_VARIABLE',
    values='VALOR_NUM',
    aggfunc='first'  # Since there are no duplicates, 'first' is sufficient
).reset_index()

# Flatten column names (remove multi-index if any)
weather_daily.columns.name = None

print(f"Weather daily shape: {weather_daily.shape}")
print(f"Number of station-date combinations: {len(weather_daily)}")
print(f"Number of weather variables: {len(weather_daily.columns) - 2}")  # Subtract CODI_ESTACIO and DATA_LECTURA
print(f"\nColumns: {list(weather_daily.columns[:10])}...")  # Show first 10 columns
weather_daily.head()


Weather daily shape: (2184, 24)
Number of station-date combinations: 2184
Number of weather variables: 22

Columns: ['CODI_ESTACIO', 'DATA_LECTURA', 'Amplitud tèrmica diària', 'Direcció de la ratxa màx. diària de vent 10 m', 'Direcció mitjana diària del vent 10 m (m. 1)', 'Evapotranspiració de referència', 'Humitat relativa mitjana diària', 'Humitat relativa màxima diària + data', 'Humitat relativa mínima diària + data', 'Irradiació solar global diària']...


,CODI_ESTACIO,DATA_LECTURA,Amplitud tèrmica diària,Direcció de la ratxa màx. diària de vent 10 m,Direcció mitjana diària del vent 10 m (m. 1),Evapotranspiració de referència,Humitat relativa mitjana diària,Humitat relativa màxima diària + data,Humitat relativa mínima diària + data,Irradiació solar global diària,...,Precipitació màxima en 30 min (diària)+ hora,Pressió atmosfèrica mitjana diària,Pressió atmosfèrica màxima diària + hora,Pressió atmosfèrica mínima diària + hora,Ratxa màxima diària del vent 10 m + hora,Temperatura mitjana diària,Temperatura mitjana diària clàssica,Temperatura màxima diària + hora,Temperatura mínima diària + hora,Velocitat mitjana diària del vent 10 m (esc.)
0,D5,2023-01-04,8.6,338.0,335.0,1.09,66.0,87.0,39.0,8.9,...,0.0,983.1,984.7,982.3,7.6,11.7,13.4,17.7,9.1,3.5
1,D5,2023-01-05,7.6,304.0,263.0,1.18,53.0,67.0,35.0,9.2,...,0.0,979.0,982.7,975.8,8.7,11.7,12.5,16.3,8.7,3.5
2,D5,2023-01-06,7.7,310.0,284.0,1.02,59.0,77.0,47.0,9.2,...,0.0,975.5,976.9,974.7,10.4,10.0,11.1,14.9,7.2,3.9
3,D5,2023-01-07,6.1,295.0,267.0,1.13,67.0,100.0,46.0,9.6,...,0.0,971.3,974.7,968.7,14.7,9.5,10.3,13.3,7.2,5.8
4,D5,2023-01-08,3.8,274.0,266.0,0.58,72.0,86.0,56.0,3.2,...,0.1,964.4,968.8,961.1,11.9,11.9,11.7,13.6,9.8,4.5


In [17]:
# Merge weather data with census sections
# Each census section gets the weather data from its assigned station (WEATHER_STATION)
gdf_daily = gdf.merge(
    weather_daily,
    left_on="WEATHER_STATION",
    right_on="CODI_ESTACIO",
    how="left"
)

print(f"Final GeoDataFrame shape: {gdf_daily.shape}")
print(f"\nNumber of census sections: {gdf_daily['SECCIO_CENSAL'].nunique()}")
print(f"\nNumber of unique dates: {gdf_daily['DATA_LECTURA'].nunique()}")
print(f"\nDate range: {gdf_daily['DATA_LECTURA'].min()} to {gdf_daily['DATA_LECTURA'].max()}")

# Show sample
gdf_daily.head(10)


Final GeoDataFrame shape: (777504, 38)

Number of census sections: 1068

Number of unique dates: 728

Date range: 2023-01-04 00:00:00 to 2024-12-31 00:00:00


,codi_districte,nom_districte,codi_barri,nom_barri,codi_aeb,codi_seccio_censal,geometria_etrs89,geometry,centroid,centroid_lat,...,Precipitació màxima en 30 min (diària)+ hora,Pressió atmosfèrica mitjana diària,Pressió atmosfèrica màxima diària + hora,Pressió atmosfèrica mínima diària + hora,Ratxa màxima diària del vent 10 m + hora,Temperatura mitjana diària,Temperatura mitjana diària clàssica,Temperatura màxima diària + hora,Temperatura mínima diària + hora,Velocitat mitjana diària del vent 10 m (esc.)
0,01,Ciutat Vella,01,el Raval,001,001,"POLYGON ((431076.9025 4581077.3095, 431058.164...","POLYGON ((2.17575 41.37827, 2.17552 41.37865, ...",POINT (2.17722 41.37432),41.374324,...,0.0,1028.5,1030.2,1027.7,4.1,13.0,13.5,16.8,10.2,0.9
1,01,Ciutat Vella,01,el Raval,001,001,"POLYGON ((431076.9025 4581077.3095, 431058.164...","POLYGON ((2.17575 41.37827, 2.17552 41.37865, ...",POINT (2.17722 41.37432),41.374324,...,0.0,1024.2,1028.0,1021.2,5.4,12.5,13.1,17.3,8.8,1.1
2,01,Ciutat Vella,01,el Raval,001,001,"POLYGON ((431076.9025 4581077.3095, 431058.164...","POLYGON ((2.17575 41.37827, 2.17552 41.37865, ...",POINT (2.17722 41.37432),41.374324,...,0.0,1020.8,1022.4,1019.9,5.8,11.8,11.9,16.0,7.7,0.9
3,01,Ciutat Vella,01,el Raval,001,001,"POLYGON ((431076.9025 4581077.3095, 431058.164...","POLYGON ((2.17575 41.37827, 2.17552 41.37865, ...",POINT (2.17722 41.37432),41.374324,...,0.0,1016.6,1020.1,1013.6,10.2,12.5,13.0,16.7,9.3,1.7
4,01,Ciutat Vella,01,el Raval,001,001,"POLYGON ((431076.9025 4581077.3095, 431058.164...","POLYGON ((2.17575 41.37827, 2.17552 41.37865, ...",POINT (2.17722 41.37432),41.374324,...,0.1,1008.9,1013.7,1005.5,6.8,14.2,14.0,16.2,11.7,1.8
5,01,Ciutat Vella,01,el Raval,001,001,"POLYGON ((431076.9025 4581077.3095, 431058.164...","POLYGON ((2.17575 41.37827, 2.17552 41.37865, ...",POINT (2.17722 41.37432),41.374324,...,0.0,1013.2,1020.7,1006.8,14.5,14.5,14.4,17.1,11.7,4.2
6,01,Ciutat Vella,01,el Raval,001,001,"POLYGON ((431076.9025 4581077.3095, 431058.164...","POLYGON ((2.17575 41.37827, 2.17552 41.37865, ...",POINT (2.17722 41.37432),41.374324,...,0.0,1023.0,1025.2,1020.0,8.6,12.2,12.6,15.7,9.4,1.6
7,01,Ciutat Vella,01,el Raval,001,001,"POLYGON ((431076.9025 4581077.3095, 431058.164...","POLYGON ((2.17575 41.37827, 2.17552 41.37865, ...",POINT (2.17722 41.37432),41.374324,...,0.0,1020.8,1023.9,1019.0,9.4,13.0,13.2,16.4,9.9,1.2
8,01,Ciutat Vella,01,el Raval,001,001,"POLYGON ((431076.9025 4581077.3095, 431058.164...","POLYGON ((2.17575 41.37827, 2.17552 41.37865, ...",POINT (2.17722 41.37432),41.374324,...,0.0,1023.1,1025.6,1020.5,7.5,12.2,12.8,16.0,9.6,1.9
9,01,Ciutat Vella,01,el Raval,001,001,"POLYGON ((431076.9025 4581077.3095, 431058.164...","POLYGON ((2.17575 41.37827, 2.17552 41.37865, ...",POINT (2.17722 41.37432),41.374324,...,0.0,1021.7,1024.5,1019.6,5.1,12.7,13.5,16.8,10.2,1.1


### Current Structure of `gdf_daily`

At this point, `gdf_daily` contains:

**Rows (Observations):**
- One row per **census section** per **date** (from 2023-01-04 to 2024-12-31)
- Total rows = Number of census sections (1068) × Number of dates in the filtered period
- Each row represents a unique combination of a census section and a date

**Columns (Variables):**

1. **Census Section Information:**
   - `SECCIO_CENSAL`: Unique identifier (format: 080193 + district + section)
   - `codi_districte`, `nom_districte`: District codes and names
   - `codi_barri`, `nom_barri`: Neighborhood codes and names
   - `codi_aeb`, `codi_seccio_censal`: Additional identifiers
   - `geometry`: Polygon geometry of the census section
   - `centroid`, `centroid_lat`, `centroid_lon`: Geographic centroids

2. **Weather Station Assignment:**
   - `WEATHER_STATION`: Assigned station (D5, X4, or X8)
   - `dist_m`: Distance to nearest weather station (in meters)
   - `CODI_ESTACIO`: Station code (same as WEATHER_STATION)

3. **Weather Variables (24 columns):**
   - `DATA_LECTURA`: Date of the weather reading
   - All 24 weather variables from the weather stations (temperature, precipitation, humidity, pressure, wind, etc.)

**Next Steps:**
We will now merge consumption, leak incidents, and socioeconomic data to enrich this dataset further.


## 7. Merge Additional Data Sources

Now we'll merge consumption, leak incidents, and socioeconomic data with the weather-enriched census sections. Each dataset needs to be aggregated appropriately to match the daily structure of `gdf_daily`.


### 7.1 Load and Merge Socioeconomic Data (IST)

The socioeconomic data (IST - Índex socioeconòmic territorial) is a static factor that we will keep constant across years (for now) for each census section. We'll load it and merge directly by `SECCIO_CENSAL`.


In [18]:
# Load socioeconomic data
socio = pd.read_parquet("clean/socio_clean.parquet")

print(f"Socioeconomic data shape: {socio.shape}")
print(f"Years: {sorted(socio['any'].unique())}")
print(f"Number of unique census sections: {socio['SECCIO_CENSAL'].nunique()}")
print(f"\nSample socioeconomic data:")
socio.head()


Socioeconomic data shape: (1068, 4)
Years: [2022]
Number of unique census sections: 1068

Sample socioeconomic data:


,any,SECCIO_CENSAL,concepte,valor
0,2022,08019301001,Índex socioeconòmic territorial,85.7
1,2022,08019301002,Índex socioeconòmic territorial,75.8
2,2022,08019301003,Índex socioeconòmic territorial,73.7
3,2022,08019301004,Índex socioeconòmic territorial,81.8
4,2022,08019301005,Índex socioeconòmic territorial,79.1


In [19]:
# Since IST is static (constant across years), we'll take one value per SECCIO_CENSAL
# Filter to get the IST value (concepte = "Índex socioeconòmic territorial")
socio_ist = socio[socio['concepte'] == 'Índex socioeconòmic territorial'].copy()

# Select SECCIO_CENSAL and valor, rename valor to ist
socio_ist = socio_ist[['SECCIO_CENSAL', 'valor']].copy()
socio_ist = socio_ist.rename(columns={'valor': 'ist'})

# If there are multiple years, take the most recent one (or first if all same)
# Group by SECCIO_CENSAL and take the first value (they should all be the same anyway)
socio_ist = socio_ist.groupby('SECCIO_CENSAL')['ist'].first().reset_index()

print(f"Socioeconomic IST shape: {socio_ist.shape}")
print(f"Number of unique census sections: {socio_ist['SECCIO_CENSAL'].nunique()}")
print(f"\nIST statistics:")
print(socio_ist['ist'].describe())
print(f"\nSample IST data:")
socio_ist.head()


Socioeconomic IST shape: (1068, 2)
Number of unique census sections: 1068

IST statistics:
count    1068.000000
mean      108.917603
std        14.578281
min        51.700000
25%       101.200000
50%       111.100000
75%       118.200000
max       138.000000
Name: ist, dtype: float64

Sample IST data:


,SECCIO_CENSAL,ist
0,08019301001,85.7
1,08019301002,75.8
2,08019301003,73.7
3,08019301004,81.8
4,08019301005,79.1


### 7.2 Load and Aggregate Consumption Data

Consumption data is split across multiple parquet files. We'll load all files, aggregate by `SECCIO_CENSAL` and `FECHA` (date), and calculate daily consumption metrics.


In [21]:
import glob
import os

# Load all consumption parquet files
consum_files = glob.glob("clean/split_consum_bcn/consum_clean_bcn_part_*.parquet")
consum_files.sort()  # Ensure consistent order

print(f"Found {len(consum_files)} consumption files")

# Load and concatenate all consumption files
consum_list = []
for file in consum_files:
    df = pd.read_parquet(file)
    consum_list.append(df)

consum = pd.concat(consum_list, ignore_index=True)

# Filter to start at 2023-01-04 (temporal overlap with weather and leaks)
date_start = pd.to_datetime('2023-01-04')
consum = consum[pd.to_datetime(consum['FECHA']) >= date_start].copy()

print(f"\nTotal consumption records (after filtering): {len(consum)}")
print(f"Date range: {consum['FECHA'].min()} to {consum['FECHA'].max()}")
print(f"Number of unique census sections: {consum['SECCIO_CENSAL'].nunique()}")
print(f"\nSample consumption data:")
consum.head()


Found 18 consumption files

Total consumption records (after filtering): 5014453
Date range: 2023-01-04 00:00:00 to 2024-12-31 00:00:00
Number of unique census sections: 621

Sample consumption data:


,POLIZA_SUMINISTRO,FECHA,CONSUMO_REAL,SECCIO_CENSAL,US_AIGUA_GEST,DATA_INST_COMP
730,VECWAVDUULZDSBOP,2023-01-04,2070,08019303025,C,2016-04-25
731,VECWAVDUULZDSBOP,2023-01-05,1938,08019303025,C,2016-04-25
732,VECWAVDUULZDSBOP,2023-01-06,4,08019303025,C,2016-04-25
733,VECWAVDUULZDSBOP,2023-01-07,53,08019303025,C,2016-04-25
734,VECWAVDUULZDSBOP,2023-01-08,7,08019303025,C,2016-04-25


In [22]:
# Aggregate to one metric per SECCIO_CENSAL per day: total consumption
consum_daily = consum.groupby(['SECCIO_CENSAL', 'FECHA'], as_index=False).agg(
    CONSUMO_TOTAL=('CONSUMO_REAL', 'sum')
)

# Rename FECHA for consistency
consum_daily = consum_daily.rename(columns={'FECHA': 'DATA_LECTURA'})

print(f"Consumption daily shape: {consum_daily.shape}")
print(f"Date range: {consum_daily['DATA_LECTURA'].min()} to {consum_daily['DATA_LECTURA'].max()}")
consum_daily.head()


Consumption daily shape: (450195, 3)
Date range: 2023-01-04 00:00:00 to 2024-12-31 00:00:00


,SECCIO_CENSAL,DATA_LECTURA,CONSUMO_TOTAL
0,08019301001,2023-01-04,4948
1,08019301001,2023-01-05,5259
2,08019301001,2023-01-06,5006
3,08019301001,2023-01-07,6301
4,08019301001,2023-01-08,5428


In [25]:
# Load leak incidents data
leaks = pd.read_parquet("clean/fuites_clean_bcn.parquet")

# Filter to date range: 2023-01-04 to 2024-12-31 (temporal overlap with weather and consumption)
date_start = pd.to_datetime('2023-01-04')
date_end = pd.to_datetime('2024-12-31')
leaks = leaks[
    (pd.to_datetime(leaks['CREATED_MENSAJE']) >= date_start) & 
    (pd.to_datetime(leaks['CREATED_MENSAJE']) <= date_end)
].copy()

print(f"Leak incidents shape (after filtering): {leaks.shape}")
print(f"Date range: {leaks['CREATED_MENSAJE'].min()} to {leaks['CREATED_MENSAJE'].max()}")
print(f"Number of unique census sections: {leaks['SECCIO_CENSAL'].nunique()}")

# Note: Not all dates will have leaks - this is normal
# When merged with gdf_daily, days without leaks will have NUM_FUITES = 0

print(f"\nSample leak data:")
leaks.head()


Leak incidents shape (after filtering): (1243, 5)
Date range: 2023-01-04 to 2024-12-30
Number of unique census sections: 428

Sample leak data:


,POLISSA_SUBM,CREATED_MENSAJE,CODIGO_MENSAJE,US_AIGUA_SUBM,SECCIO_CENSAL
0,KWHZ5UG2ZKENUFC2,2023-12-03,FUITA,DOMÈSTIC,08019305059
1,GVXPU34GVXQUIWFK,2023-08-10,FUITA,DOMÈSTIC,08019310139
2,GVXPU34GVXQUIWFK,2023-06-10,FUITA,DOMÈSTIC,08019310139
3,I7GGTJ6C6FMR5ARW,2024-09-06,FUITA,DOMÈSTIC,08019302087
4,I7GGTJ6C6FMR5ARW,2024-11-13,FUITA,DOMÈSTIC,08019302087


In [ ]:
# Aggregate leaks by SECCIO_CENSAL and CREATED_MENSAJE (date)
# Count number of leaks per day per census section
leaks_daily = leaks.groupby(['SECCIO_CENSAL', 'CREATED_MENSAJE']).agg({
    'CODIGO_MENSAJE': 'count',  # Count of leaks
    'POLISSA_SUBM': 'nunique'  # Number of unique contracts with leaks
}).reset_index()

# Rename columns
leaks_daily.columns = ['SECCIO_CENSAL', 'DATA_LECTURA', 'NUM_FUITES', 'NUM_POLISSES_FUITES']

print(f"Leaks daily shape: {leaks_daily.shape}")
print(f"Date range: {leaks_daily['DATA_LECTURA'].min()} to {leaks_daily['DATA_LECTURA'].max()}")
leaks_daily.head()


### 7.4 Merge All Data Sources

Now we'll merge consumption, leaks, and socioeconomic data with the weather-enriched `gdf_daily` GeoDataFrame.


In [ ]:
# Step 1: Merge socioeconomic data (IST) - static factor
# Merge on SECCIO_CENSAL only (no date needed since it's constant)
gdf_daily = gdf_daily.merge(
    socio_ist,
    on='SECCIO_CENSAL',
    how='left'
)

print(f"After IST merge: {gdf_daily.shape}")
print(f"IST records matched: {gdf_daily['ist'].notna().sum()}")

# Step 2: Merge consumption data
# Merge on SECCIO_CENSAL and DATA_LECTURA (date)
gdf_daily = gdf_daily.merge(
    consum_daily,
    on=['SECCIO_CENSAL', 'DATA_LECTURA'],
    how='left'
)

print(f"\nAfter consumption merge: {gdf_daily.shape}")
print(f"Consumption records matched: {gdf_daily['CONSUMO_TOTAL'].notna().sum()}")

# Step 3: Merge leak incidents data
# Merge on SECCIO_CENSAL and DATA_LECTURA (date)
gdf_daily = gdf_daily.merge(
    leaks_daily,
    on=['SECCIO_CENSAL', 'DATA_LECTURA'],
    how='left'
)

# Fill NaN with 0 for leak counts (no leaks = 0)
gdf_daily['NUM_FUITES'] = gdf_daily['NUM_FUITES'].fillna(0).astype(int)
gdf_daily['NUM_POLISSES_FUITES'] = gdf_daily['NUM_POLISSES_FUITES'].fillna(0).astype(int)

print(f"\nAfter leaks merge: {gdf_daily.shape}")
print(f"Days with leaks: {(gdf_daily['NUM_FUITES'] > 0).sum()}")

print(f"\nFinal columns: {len(gdf_daily.columns)}")
print(f"\nSample of final merged data:")
gdf_daily[['SECCIO_CENSAL', 'DATA_LECTURA', 'WEATHER_STATION', 
           'Temperatura mitjana diària', 'ist', 'CONSUMO_TOTAL', 'NUM_FUITES']].head(10)
